# Importing and loading necessary libraries

In [308]:
import numpy as np
import cv2
import random
import os
import math

from imutils import paths

from keras.models import Sequential,Input,Model,load_model
from keras.preprocessing.image import ImageDataGenerator,img_to_array
from keras.layers import Input,Conv2D,Dense,MaxPool2D,Dropout,Flatten,MaxPooling2D,GlobalAveragePooling2D,BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras import applications
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.resnet50 import ResNet50
# from keras.applications.inception_v3 import InceptionV3
from keras.layers.core import Activation
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, precision_recall_fscore_support, average_precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

import warnings 
warnings.filterwarnings("ignore",category=UserWarning)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Check the compute 

In [247]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [248]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11548101675370414562]

# Load the data

In [249]:
# from google.colab import drive
# drive.mount('/content/drive')

In [250]:
# path = "/content/drive/My Drive/Colab Notebooks/VIT_FDP_CV/data/food_5k"
# !ls "/content/drive/My Drive/Colab Notebooks/VIT_FDP_CV/data/food_5k"
# !ls "/content/drive/My Drive/Colab Notebooks/VIT_FDP_CV/data/food_5k/train"

In [251]:
PATH = "/Users/noor-4785/Desktop/food_5k/"
# PATH = "/content/drive/My Drive/Colab Notebooks/VIT_FDP_CV/data/food_5k/"

# Initialize hyper parameters

In [71]:
TRAIN_DATA_PATH = PATH + "train"
TEST_DATA_PATH = PATH + "test"
VALID_DATA_PATH= PATH + "valid"

BATCH_SIZE = 32
IMAGE_SIZE = 48

# CLASS_MODE = 'binary'
CLASS_MODE = 'categorical'


EPOCHS = 1

TARGET_NAMES=['food','notfood']

# TIMESTAMP = datetime.now().strftime("%d-%m-%Y__%H-%M-%S")



# Prepare Data

In [72]:
image_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# For Data Augumentattion

# image_data_generator=keras_image_preprocessor.ImageDataGenerator(
#         rescale=1./255,
#         rotation_range=10,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         shear_range=0.1,
#         zoom_range=0.1,
#         horizontal_flip=True,
#         #validation_split=0.1
#     )

# test_image_generator=keras_image_preprocessor.ImageDataGenerator(rescale=1./255)

In [73]:
# Use the above data generator for data augmentation in each of training, validation and test data sets

train_gen = image_data_generator.flow_from_directory(
    TRAIN_DATA_PATH,
    color_mode='rgb',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE,
    shuffle=True,
    #subset="training"
)

validation_gen = image_data_generator.flow_from_directory(
    VALID_DATA_PATH,
    color_mode='rgb',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE,
    shuffle=True,
    #subset="validation"
)

test_gen = test_image_generator.flow_from_directory(
    TEST_DATA_PATH,
    color_mode='rgb',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE,
    shuffle=False,
)





Found 3000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [74]:
print(train_gen.class_indices)
no_of_classes = len(train_gen.class_indices)
print(no_of_classes)


{'food': 0, 'notfood': 1}
2


# Transfer learning

In [75]:
base_model=ResNet50(weights='imagenet',include_top=False, input_tensor=Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3)))

In [76]:
# base_model.summary()

In [77]:
#Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

In [78]:
model= Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dense(no_of_classes,activation='softmax'))

In [79]:
model.compile(loss='categorical_crossentropy',optimizer="Adam", metrics=['accuracy'])

In [81]:
# model.summary()

In [82]:
# esc = [EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5, verbose=1, mode='max', restore_best_weights=True)]


In [83]:
H = model.fit_generator(
                    generator=train_gen,
                    steps_per_epoch=math.ceil(train_gen.samples/train_gen.batch_size),
                    validation_data=validation_gen,
                    validation_steps=math.ceil(validation_gen.samples/validation_gen.batch_size),
                    epochs=2,
                    # callbacks=esc
                )


Epoch 1/2
94/94 [==============================] - 97s 1s/step - loss: 1.3780 - acc: 0.8094 - val_loss: 0.4595 - val_acc: 0.8800
Epoch 2/2
94/94 [==============================] - 87s 931ms/step - loss: 0.2305 - acc: 0.9039 - val_loss: 0.3874 - val_acc: 0.8990


In [84]:
# Plot training & validation accuracy values
plt.plot(H.history['acc'])
plt.plot(H.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'valid'], loc='upper left')
plt.show()

In [ ]:
# model.save('Resnet50_transfer_learning.h5')

# Fine tuning

In [85]:
# Unfreeze only the last few layers
N_LAYERS_TO_FREEZE = 20

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [86]:

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])


In [87]:
H_tune = model.fit_generator(
                    generator=train_gen,
                    steps_per_epoch=math.ceil(train_gen.samples/train_gen.batch_size),
                    validation_data=validation_gen,
                    validation_steps=math.ceil(validation_gen.samples/validation_gen.batch_size),
                    epochs=2,
                    # callbacks=esc
                )


Epoch 1/2
94/94 [==============================] - 75s 799ms/step - loss: 0.1219 - acc: 0.9527 - val_loss: 0.3874 - val_acc: 0.8990
Epoch 2/2
94/94 [==============================] - 76s 811ms/step - loss: 0.1156 - acc: 0.9563 - val_loss: 0.3874 - val_acc: 0.8990


In [ ]:
# Plot training & validation accuracy values
plt.plot(H_tune.history['acc'])
plt.plot(H_tune.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'valid'], loc='upper left')
plt.show()

In [88]:
# model.save('Resnet50_finetuned.h5')

# Validation

In [89]:
test_score = model.evaluate_generator(generator=test_gen,steps=math.ceil(test_gen.samples/test_gen.batch_size))   
print("loss : ",test_score[0])   
print("accuracy : ",test_score[1])
test_gen.reset()

loss :  0.441912941634655
accuracy :  0.888


In [90]:
# get the predictions
preds = model.predict_generator(generator=test_gen,steps=math.ceil(test_gen.samples/test_gen.batch_size),verbose=1)
preds= np.argmax(preds,axis=1)
print("---------------------- Confusion Matrix ----------------------")
print(confusion_matrix(test_gen.classes,preds))
print("----------------------- Classification Report --------------------")
print(classification_report(test_gen.classes,preds,target_names=TARGET_NAMES)) 


32/32 [==============================] - 27s 837ms/step
---------------------- Confusion Matrix ----------------------
[[460  40]
 [ 72 428]]
----------------------- Classification Report --------------------
              precision    recall  f1-score   support

        food       0.86      0.92      0.89       500
     notfood       0.91      0.86      0.88       500

   micro avg       0.89      0.89      0.89      1000
   macro avg       0.89      0.89      0.89      1000
weighted avg       0.89      0.89      0.89      1000



# Prediction from file path

In [301]:
# image_path = "/Users/noor-4785/Desktop/food_5k/test/food/1_147.jpg"
image_path = "/Users/noor-4785/Desktop/food_5k/train/notfood/0_400.jpg"

In [302]:
image = cv2.imread(image_path,cv2.IMREAD_COLOR)
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
image = img_to_array(image)
image = preprocess_input(image)
image = np.expand_dims(image, axis=0)
image.shape




#     image_bytes = image.file.read()
#     img_array = np.fromstring(image_bytes, np.uint8)
#     image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
#     image = cv2.resize(image, constants.IMAGE_SIZE)
#     image = img_to_array(image)/255
#     image = np.expand_dims(image, axis=0)
#     return image

(1, 48, 48, 3)

In [303]:
model.predict(image)[0]

array([0.03174352, 0.9682565 ], dtype=float32)

In [304]:
model.predict_classes(image)[0]

1

In [305]:
TARGET_NAMES[model.predict_classes(image)[0]]

'notfood'

# Prediction from collab

In [ ]:
from google.colab import files
uploaded = files.upload()
image_bytes = list(uploaded.values())[0]
img_array = np.fromstring(image_bytes, np.uint8)
image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
image = img_to_array(image)
image = preprocess_input(image)
image = np.expand_dims(image, axis=0)
print(image.shape)
result = model.predict_classes(image)[0]
print("Model Output", TARGET_NAMES[result])